# DA - March Machine Learning Mania 2017

## 2017 Prediction Results

Using the PCBR stats from 2002 until 2015 as training for the model.  
PCBR from 2016 as validation/test.
Using PCBR stats from 2017 as actual data.  
Using the 68 teams with their bracket names for 2017.  
Method: Logistic regression  

In [1]:
import numpy as np
import pandas as pd
# LR
from sklearn import linear_model, metrics
# NN
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
from keras.regularizers import l2, l1
from sklearn.preprocessing import StandardScaler

Using TensorFlow backend.


In [2]:
teams_df = pd.read_csv('dataset/Teams2017bracket.csv')
data_df = pd.read_csv('dataset/All_PCBR_For_2002_2016_Matchup.csv')
stats_df = pd.read_csv('dataset/kenpom/PCBR_2017.csv')

In [3]:
DataX = data_df.drop(['WL','Season'], axis=1).values
DataY = data_df.WL.ravel()
# range goes until 2016 not including (until 2015 including)
train = data_df.loc[data_df.Season.isin(range(2002, 2017))].drop(['WL','Season'], axis=1).index.values # use 2002-2015 as training
test  = data_df.loc[data_df.Season == 2016].WL.index.ravel() # use 2016 as test


'''
# LR
logi = linear_model.LogisticRegression()
logi.fit(DataX[train], DataY[train])
#pred = logi.predict_proba(DataX[test])[:, 1] # get just the second array
'''

# NN
stdScaler = StandardScaler()
X_train_scaled = stdScaler.fit_transform(DataX[train])
X_test_scaled = stdScaler.transform(DataX[test])
model = Sequential()
model.add(Dense(2560, input_dim=DataX[train].shape[1], init='normal', activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(18, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(18, activation='relu'))
model.add(Dense(1, init='normal', activation='sigmoid'))
sgd = SGD(lr=0.1, momentum=0.1, decay=0.0, nesterov=False)
model.compile(loss='binary_crossentropy', optimizer='sgd')
hist = model.fit(X_train_scaled, DataY[train], nb_epoch=20, batch_size=32, verbose=2, validation_split=0.1)
##pred = model.predict(X_test_scaled)[:, 0]

#print("Log loss: {}".format(metrics.log_loss(DataY[test], pred)))

Train on 69051 samples, validate on 7673 samples
Epoch 1/20
18s - loss: 0.5977 - val_loss: 0.5063
Epoch 2/20
18s - loss: 0.5134 - val_loss: 0.4978
Epoch 3/20
16s - loss: 0.5038 - val_loss: 0.4992
Epoch 4/20
17s - loss: 0.4993 - val_loss: 0.5002
Epoch 5/20
16s - loss: 0.4958 - val_loss: 0.5010
Epoch 6/20
16s - loss: 0.4948 - val_loss: 0.5048
Epoch 7/20
16s - loss: 0.4938 - val_loss: 0.5031
Epoch 8/20
17s - loss: 0.4910 - val_loss: 0.5072
Epoch 9/20
17s - loss: 0.4897 - val_loss: 0.5092
Epoch 10/20
17s - loss: 0.4900 - val_loss: 0.5094
Epoch 11/20
17s - loss: 0.4882 - val_loss: 0.5122
Epoch 12/20
17s - loss: 0.4869 - val_loss: 0.5134
Epoch 13/20
17s - loss: 0.4856 - val_loss: 0.5140
Epoch 14/20
18s - loss: 0.4862 - val_loss: 0.5211
Epoch 15/20
17s - loss: 0.4851 - val_loss: 0.5183
Epoch 16/20
16s - loss: 0.4853 - val_loss: 0.5214
Epoch 17/20
16s - loss: 0.4852 - val_loss: 0.5224
Epoch 18/20
16s - loss: 0.4839 - val_loss: 0.5240
Epoch 19/20
16s - loss: 0.4841 - val_loss: 0.5251
Epoch 20/2

In [4]:
DataX[test].shape

(5436, 18)

In [5]:
curr_teams_df = pd.merge(teams_df, stats_df, how='inner', left_on='Team_Name', right_on='Team')
print(curr_teams_df.shape)
curr_teams_df.head()

(68, 16)


,Team_Id,Team_Name,Bracket_Name,Rank,Team,Conf,W-L,AdjEM,AdjO,AdjD,AdjT,Luck,SOS AdjEM,SOS OppO,SOS OppD,NCSOS AdjEM
0,1112,Arizona,Arizona,18,Arizona,P12,32-4,23.60,118.5,94.9,66.0,0.062,6.87,108.8,101.9,1.77
1,1116,Arkansas,Arkansas,36,Arkansas,SEC,26-10,16.07,116.3,100.3,69.6,0.086,8.31,108.8,100.5,-1.37
2,1124,Baylor,Baylor,10,Baylor,B12,27-7,25.25,118.3,93.0,63.4,0.009,12.62,112.0,99.3,1.68
3,1137,Bucknell,Bucknell,76,Bucknell,Pat,26-9,9.36,110.0,100.7,69.8,0.006,-3.57,101.7,105.3,0.90
4,1139,Butler,Butler,24,Butler,BE,25-8,21.15,117.9,96.8,65.6,0.004,10.40,110.8,100.4,5.22


In [6]:
CTData = curr_teams_df[['Team_Id','Bracket_Name','AdjEM','AdjO','AdjD','AdjT','Luck','SOS AdjEM','SOS OppO','SOS OppD','NCSOS AdjEM']].copy()
CTData['dkey'] = 1

TTData = pd.merge(CTData, CTData, how='inner', left_on='dkey', right_on='dkey')
TTData.head()

,Team_Id_x,Bracket_Name_x,AdjEM_x,AdjO_x,AdjD_x,AdjT_x,Luck_x,SOS AdjEM_x,SOS OppO_x,SOS OppD_x,...,Bracket_Name_y,AdjEM_y,AdjO_y,AdjD_y,AdjT_y,Luck_y,SOS AdjEM_y,SOS OppO_y,SOS OppD_y,NCSOS AdjEM_y
0,1112,Arizona,23.6,118.5,94.9,66.0,0.062,6.87,108.8,101.9,...,Arizona,23.60,118.5,94.9,66.0,0.062,6.87,108.8,101.9,1.77
1,1112,Arizona,23.6,118.5,94.9,66.0,0.062,6.87,108.8,101.9,...,Arkansas,16.07,116.3,100.3,69.6,0.086,8.31,108.8,100.5,-1.37
2,1112,Arizona,23.6,118.5,94.9,66.0,0.062,6.87,108.8,101.9,...,Baylor,25.25,118.3,93.0,63.4,0.009,12.62,112.0,99.3,1.68
3,1112,Arizona,23.6,118.5,94.9,66.0,0.062,6.87,108.8,101.9,...,Bucknell,9.36,110.0,100.7,69.8,0.006,-3.57,101.7,105.3,0.90
4,1112,Arizona,23.6,118.5,94.9,66.0,0.062,6.87,108.8,101.9,...,Butler,21.15,117.9,96.8,65.6,0.004,10.40,110.8,100.4,5.22


In [7]:
TTData.head()

,Team_Id_x,Bracket_Name_x,AdjEM_x,AdjO_x,AdjD_x,AdjT_x,Luck_x,SOS AdjEM_x,SOS OppO_x,SOS OppD_x,...,Bracket_Name_y,AdjEM_y,AdjO_y,AdjD_y,AdjT_y,Luck_y,SOS AdjEM_y,SOS OppO_y,SOS OppD_y,NCSOS AdjEM_y
0,1112,Arizona,23.6,118.5,94.9,66.0,0.062,6.87,108.8,101.9,...,Arizona,23.60,118.5,94.9,66.0,0.062,6.87,108.8,101.9,1.77
1,1112,Arizona,23.6,118.5,94.9,66.0,0.062,6.87,108.8,101.9,...,Arkansas,16.07,116.3,100.3,69.6,0.086,8.31,108.8,100.5,-1.37
2,1112,Arizona,23.6,118.5,94.9,66.0,0.062,6.87,108.8,101.9,...,Baylor,25.25,118.3,93.0,63.4,0.009,12.62,112.0,99.3,1.68
3,1112,Arizona,23.6,118.5,94.9,66.0,0.062,6.87,108.8,101.9,...,Bucknell,9.36,110.0,100.7,69.8,0.006,-3.57,101.7,105.3,0.90
4,1112,Arizona,23.6,118.5,94.9,66.0,0.062,6.87,108.8,101.9,...,Butler,21.15,117.9,96.8,65.6,0.004,10.40,110.8,100.4,5.22


In [8]:
TTData.rename(columns={'AdjEM_x':'T1_AdjEM',
                     'AdjO_x':'T1_AdjO',
                     'AdjD_x':'T1_AdjD',
                     'AdjT_x':'T1_AdjT',
                     'Luck_x':'T1_Luck',
                     'SOS AdjEM_x':'T1_SOSADjEM',
                     'SOS OppO_x':'T1_SOSOppO',
                     'SOS OppD_x':'T1_SOSOppD',
                     'NCSOS AdjEM_x':'T1_NCSOSAdjEM',
                     'AdjEM_y':'T2_AdjEM',
                     'AdjO_y':'T2_AdjO',
                     'AdjD_y':'T2_AdjD',
                     'AdjT_y':'T2_AdjT',
                     'Luck_y':'T2_Luck',
                     'SOS AdjEM_y':'T2_SOSADjEM',
                     'SOS OppO_y':'T2_SOSOppO',
                     'SOS OppD_y':'T2_SOSOppD',
                     'NCSOS AdjEM_y':'T2_NCSOSAdjEM'}, inplace=True)
TTData.head()

,Team_Id_x,Bracket_Name_x,T1_AdjEM,T1_AdjO,T1_AdjD,T1_AdjT,T1_Luck,T1_SOSADjEM,T1_SOSOppO,T1_SOSOppD,...,Bracket_Name_y,T2_AdjEM,T2_AdjO,T2_AdjD,T2_AdjT,T2_Luck,T2_SOSADjEM,T2_SOSOppO,T2_SOSOppD,T2_NCSOSAdjEM
0,1112,Arizona,23.6,118.5,94.9,66.0,0.062,6.87,108.8,101.9,...,Arizona,23.60,118.5,94.9,66.0,0.062,6.87,108.8,101.9,1.77
1,1112,Arizona,23.6,118.5,94.9,66.0,0.062,6.87,108.8,101.9,...,Arkansas,16.07,116.3,100.3,69.6,0.086,8.31,108.8,100.5,-1.37
2,1112,Arizona,23.6,118.5,94.9,66.0,0.062,6.87,108.8,101.9,...,Baylor,25.25,118.3,93.0,63.4,0.009,12.62,112.0,99.3,1.68
3,1112,Arizona,23.6,118.5,94.9,66.0,0.062,6.87,108.8,101.9,...,Bucknell,9.36,110.0,100.7,69.8,0.006,-3.57,101.7,105.3,0.90
4,1112,Arizona,23.6,118.5,94.9,66.0,0.062,6.87,108.8,101.9,...,Butler,21.15,117.9,96.8,65.6,0.004,10.40,110.8,100.4,5.22


In [9]:
GTData = TTData.loc[TTData.Team_Id_x < TTData.Team_Id_y]
GTData.shape

(2278, 23)

In [10]:
PRData = GTData.drop(['Team_Id_x','Bracket_Name_x','Team_Id_y','Bracket_Name_y','dkey'], axis=1)

# LR
#res = logi.predict_proba(PRData)[:, 1] # get just the second array

# NN
X_test_scaled = stdScaler.transform(PRData)
res = model.predict(X_test_scaled)[:, 0]

In [11]:
res.size

2278

In [12]:
FData = GTData[['Team_Id_x','Bracket_Name_x','Team_Id_y','Bracket_Name_y']].copy()
FData['Prediction'] = res
FData.head()

,Team_Id_x,Bracket_Name_x,Team_Id_y,Bracket_Name_y,Prediction
1,1112,Arizona,1116,Arkansas,0.604869
2,1112,Arizona,1124,Baylor,0.471982
3,1112,Arizona,1137,Bucknell,0.792402
4,1112,Arizona,1139,Butler,0.603954
5,1112,Arizona,1153,Cincinnati,0.600465


In [13]:
FData.to_csv('dataset/Prediction_2017_results.csv', index=False)